In [186]:
from dotenv import load_dotenv
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from openai import OpenAI
import yfinance as yf
import json
import requests
import os

client = OpenAI()


# openai assistant playground 에서는 실행 결과를 직접 붙여 넣기 해야 한다
def get_ticker(inputs):
    # return 'CRM'
    company_name = inputs['company_name']
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(f'ticker symbol for {company_name}')


def get_income_statement(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.income_stmt
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


def get_balance_sheet(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.balance_sheet
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


def get_daily_stock_performance(inputs):
    ticker = inputs['ticker']
    company = yf.Ticker(ticker)
    pd_frame = company.history(period='3mo')
    json_obj = pd_frame.to_json(
        orient='records',  # Output format: list of records
        date_format='iso',  # Use ISO date format
        double_precision=2,  # Precision of floating-point numbers
        force_ascii=False,  # Preserve non-ASCII characters
        date_unit='ms'  # Timestamp unit in milliseconds
    )

    return json.dumps(json_obj)


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# open ai assistant 생성
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not",
#     model="gpt-4-turbo",
#     tools=functions
# )

assistant_id = 'asst_xJDilHdQyVS7L9jn8xODLqlh'

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy"
        }
    ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

load_dotenv()


def submit_tools(thread_id, run_id, tool_info_list):
    url = f'https://api.openai.com/v1/threads/{thread_id}/runs/{run_id}/submit_tool_outputs'
    headers = {
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
        "Content-Type": "application/json",
        "OpenAI-Beta": "assistants=v2"
    }

    data = {
        "tool_outputs": tool_info_list
    }

    # content-type 이 application/json 일 경우 
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code != 200:
        print(response.text)
        
    print(response.status_code)


def chk_status(thread_id, run_id) -> dict:
    chk_url = f'https://api.openai.com/v1/threads/{thread_id}/runs/{run_id}'

    chk_headers = {
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
        "OpenAI-Beta": "assistants=v2"
    }
    
    # content-type 이 application/json 일 경우 
    result = requests.get(chk_url, headers=chk_headers)
    return json.loads(result.text)


In [187]:

import openai


def exec_tool(thread_id: str, run_id: str, tool_info_list):
    # run.status 가 requires_action 일 경우 tool_calls 에 LLM 이 답변을 기다 리는 정보가 있으 므로 
    # https://platform.openai.com/docs/api-reference/runs/submitToolOutputs 에 지정된 형식 으로 정보를 전달 하면 됨
    
    tool_result_list = []
    for tool_info in tool_info_list:
        tool_id = tool_info['id']
        func_info = tool_info['function']
        
        tool_call_result = globals()[func_info['name']](json.loads(func_info['arguments']))
        tool_result_list.append({
            "tool_call_id": tool_id,
            "output": tool_call_result
        })
    
    print('tool_result_dict : ', tool_result_list)
    submit_tools(thread_id, run_id, tool_result_list)

def chk_status_and_run(thread_id: str, run_id: str) -> str:
    status_json = chk_status(thread_id, run_id)
    status_str = status_json['status']
    if status_str == 'completed':
        print('run completed')
        messages = client.beta.threads.messages.list(
            thread_id=thread_id
        )
        print(messages)
    else:
        if status_str == 'requires_action':
            print('exec_tool')
            tool_info_list = status_json['required_action']['submit_tool_outputs']['tool_calls']
            print('tool_info_list : ', tool_info_list)
            
            exec_tool(thread_id, run_id, tool_info_list)
        else:
            print('current status : ', status_str)
    
    return status_str


In [188]:
chk = ''

while chk != 'completed':
    chk = chk_status_and_run(thread.id, run.id)

exec_tool
tool_info_list :  [{'id': 'call_Q0NFC6PsiaC6YKfrj6UWfZez', 'type': 'function', 'function': {'name': 'get_ticker', 'arguments': '{"company_name":"Salesforce"}'}}]
tool_result_dict :  [{'tool_call_id': 'call_Q0NFC6PsiaC6YKfrj6UWfZez', 'output': "Salesforce is the world's #1 customer relationship management (CRM) platform. Learn More. Mobile Site Search. Overview. About; Stock Information. Stock Chart; ... Current Stock quote. Historical stock quote. Investment calculator. Investment Calculator Options. Amount ($) Start Date: End Date: Compare to: S&P 500. Nasdaq 100. Dow 30. Other. Stock analysis for Salesforce Inc (CRM:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Dan Romanoff May 30, 2024. We are lowering our fair value estimate for wide-moat Salesforce to $285 per share from $300 after the company reported lower-than-expected fiscal 2025 first-quarter ... 35 Wall Street analysts have issued 1 year target prices 